<a href="https://colab.research.google.com/github/javajaewook/ML_Team_SMILE/blob/master/RIOT_gamedata_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install riotwatcher # library for riot api

     |████████████████████████████████| 51kB 2.1MB/s 


In [0]:
 # 발급받은 api key, 발급 후 24시간 후 파기되므로, 24시간후 재발급 해야함
api_key = 'RGAPI-b2d101f8-c07c-47f9-b436-7b49885f30c1'

In [3]:
from riotwatcher import LolWatcher
import time

# 소환사 정보
watcher = LolWatcher(api_key)
me = watcher.summoner.by_name('kr', '광진구대가리')
print(me)

{'id': 'YHWH6ka9_8xKX8u5XAUw4B_2lblxf1hJGJxJ0G-es2ZT27w', 'accountId': 'xZzggA0nu4cNHOiJpB4OsNByHK-eOEWPjwGuVUSytub0etd6fdmnURth', 'puuid': 'IItKt9jHYizB6mWZm6CSp9mAXOolZKmHBL9M3VP86H3k8FoGV-lTdFSaTgu292qnz-5lKdJrl4vOJw', 'name': '광진구대가리', 'profileIconId': 4491, 'revisionDate': 1589972969000, 'summonerLevel': 14}


In [6]:
summoner = "https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/" +'광진구대가리' +'?api_key=' + api_key
r = requests.get(summoner)
r.json()['id'] #소환사의 고유 id

'YHWH6ka9_8xKX8u5XAUw4B_2lblxf1hJGJxJ0G-es2ZT27w'

In [7]:
tier_url = "https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/" + r.json()['id'] +'?api_key=' + api_key
r2  = requests.get(tier_url)
r2.json()

{'status': {'message': 'Forbidden', 'status_code': 403}}

In [5]:
import requests
import pandas as pd

master = 'https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
r = requests.get(master) # 다이아 데이터 호출
league_df = pd.DataFrame(r.json())

league_df.reset_index(inplace=True) #수집한 데이터 index정리
league_entries_df = pd.DataFrame(dict(league_df['entries'])).T  # dict구조로 되어 있는 entries컬럼 풀어주기
league_df = pd.concat([league_df, league_entries_df], axis=1)  # 열끼리 결합

league_df = league_df.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank'], axis=1)
league_df.info()
league_df.to_csv('LoL_gameData_master.csv',index=False,encoding = 'cp949')#중간저장

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1411 entries, 0 to 1410
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tier          1411 non-null   object
 1   summonerId    1411 non-null   object
 2   summonerName  1411 non-null   object
 3   leaguePoints  1411 non-null   object
 4   wins          1411 non-null   object
 5   losses        1411 non-null   object
 6   veteran       1411 non-null   object
 7   inactive      1411 non-null   object
 8   freshBlood    1411 non-null   object
 9   hotStreak     1411 non-null   object
dtypes: object(10)
memory usage: 110.4+ KB


In [38]:
league_df.head() # 

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,MASTER,0NCoABMR9-2fzsZAQERFmQOCZFRVdG9uGZ_hwySWuBwOVAA,이하사TV,76,432,422,False,False,False,False
1,MASTER,nuoSYLMUApDVKVyM3fAT0MjZ1PmgtdotQ11oanNm9IJ-YUY,미드생태계교란종,38,328,311,False,False,True,False
2,MASTER,MtaQ73iv5fqmd6rVEmxEXF1Tzo74L0S9KXcmKd8w-lhZxKY,vt77,187,178,162,False,False,False,True
3,MASTER,FwijOhTC69fXputPGtLND3JRUnoXSNvdWDMRea59jm3fbsc,당신을 사냥,171,440,366,False,False,True,False
4,MASTER,t_xHtlCrBtjpY6WFX2V4_1XXG7KBhvDieTq51-ucDDOm3lQ,EMENES,78,348,336,False,False,True,False


In [0]:
for i in range(len(league_df)): # 해당 리그에서 수집해온 소환사 data
  try:
    summoner = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key
    r = requests.get(summoner)

    while r.status_code == 429: # 단기간 너무 많은 데이터를 요청하면 거부 -> 시간 기다렸다가 다시 요청
      time.sleep(5)
      summoner = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key
      r = requests.get(summoner)
    
    account_id = r.json()['accountId'] # 받아온 계정 고유 ID를 league_df 맨 뒤 속성에 덮어 씌우기
    league_df.iloc[i,-1] = account_id
  except:
    pass


In [0]:
match_info_df = pd.DataFrame()
season = str()
for i in range(len(league_df)):
    try:
        match0 = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + league_df['hotStreak'].iloc[i]  +'?season=' + season + '&api_key=' + api_key
        r = requests.get(match0)
        
        while r.status_code == 429:
            time.sleep(5)
            match0 = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + league_df['hotStreak'].iloc[i]  +'?season=' + season + '&api_key=' + api_key
            r = requests.get(match0)
            
        match_info_df = pd.concat([match_info_df, pd.DataFrame(r.json()['matches'])])
    
    except:
        print(i)

8
10
16
26
